<a href="https://colab.research.google.com/github/worldhello23/Chrome-yt-extension-/blob/master/forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv("train.csv")
df_train

,district_id,age_bin_id,gender_id,as_of_date_id,count
0,0,0,0,117,1384
1,0,0,1,117,1432
2,1,0,0,117,1145
3,1,0,1,117,1166
4,2,0,0,117,237
...,...,...,...,...,...
14611,18,5,1,42,890
14612,19,5,0,42,6794
14613,19,5,1,42,2998
14614,20,5,0,42,1210


In [ ]:
df_test = pd.read_csv("test.csv")
df_test

,ID,district_id,age_bin_id,gender_id,as_of_date_id
0,0,0,0,0,130
1,1,0,0,1,130
2,2,1,0,0,130
3,3,1,0,1,130
4,4,2,0,0,130
...,...,...,...,...,...
3271,3271,18,5,1,118
3272,3272,19,5,0,118
3273,3273,19,5,1,118
3274,3274,20,5,0,118


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load your data (replace 'your_data.csv' with your actual file path)
df = df_train

# Assuming 'district_id' and 'gender_id' are the only categorical features
categorical_features = ['district_id', 'gender_id']

# One-Hot Encoding
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_columns = encoder.fit_transform(df[categorical_features])
encoded_df = pd.DataFrame(encoded_columns.toarray(), columns=encoder.get_feature_names_out(categorical_features))

# Combine the encoded features with the rest of the dataframe
df = pd.concat([df.drop(categorical_features, axis=1), encoded_df], axis=1)

# Features (X) and Target Variable (y)
X = df.drop('count', axis=1)
y = df['count']

# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
model = LinearRegression()  # You can use other models like RandomForestRegressor, etc.
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

# # Make Predictions on New Data (assuming new_data is a DataFrame with the same columns as X_test)
# new_data_encoded = encoder.transform(new_data[categorical_features])
# new_data_encoded_df = pd.DataFrame(new_data_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_features))
# new_data = pd.concat([new_data.drop(categorical_features, axis=1), new_data_encoded_df], axis=1)
# predictions = model.predict(new_data)


Mean Squared Error on Test Set: 2563067.5124578676


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load your data (replace 'your_data.csv' with your actual file path)
# For example:
df = df_train # Replace with your file path

# Assuming 'district_id' and 'gender_id' are the only categorical features
categorical_features = ['district_id', 'gender_id']

# One-Hot Encoding
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_columns = encoder.fit_transform(df[categorical_features])
encoded_df = pd.DataFrame(encoded_columns.toarray(), columns=encoder.get_feature_names_out(categorical_features))

# Combine the encoded features with the rest of the dataframe
df = pd.concat([df.drop(categorical_features, axis=1), encoded_df], axis=1)

# Features (X) and Target Variable (y)
X = df.drop('count', axis=1)
y = df['count']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11),
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [True, False],
}

random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    cv=5,  # 5-fold cross-validation
    scoring='neg_mean_absolute_error',
    random_state=42
)

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate Best Model
y_pred = best_model.predict(X_test)
mse = mean_absolute_error(y_test, y_pred)
# Splitting Data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Model Training (Random Forest Regressor)
# model = RandomForestRegressor(random_state=42)
# model.fit(X_train, y_train)

# # Model Evaluation
# y_pred = model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error on Test Set: {mse}')

# # Make Predictions on New Data (assuming new_data is a DataFrame with the same columns as X_test)
# # new_data_encoded = encoder.transform(new_data[categorical_features])
# # new_data_encoded_df = pd.DataFrame(new_data_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_features))
# # new_data = pd.concat([new_data.drop(categorical_features, axis=1), new_data_encoded_df], axis=1)
# # predictions = model.predict(new_data)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

In [ ]:
df_test_encoded = encoder.transform(df_test[categorical_features])
df_test_encoded_df = pd.DataFrame(df_test_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_features))
df_test = pd.concat([df_test.drop(categorical_features, axis=1), df_test_encoded_df], axis=1)


In [ ]:
df_test

,age_bin_id,as_of_date_id,district_id_0,district_id_1,district_id_2,district_id_3,district_id_4,district_id_5,district_id_6,district_id_7,...,district_id_13,district_id_14,district_id_15,district_id_16,district_id_17,district_id_18,district_id_19,district_id_20,gender_id_0,gender_id_1
ID,,,,,,,,,,,,,,,,,,,,,
0,0,130,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,130,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,130,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,130,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,130,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,5,118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3272,5,118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3273,5,118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
#
# df_test = df_test.set_index("ID")
predictions = best_model.predict(df_test)


In [ ]:
predictions

array([1376.16, 1424.74, 1134.22, ..., 2597.86, 1336.83,  741.92])

In [ ]:
dh = pd.DataFrame( predictions,df_test.index)
dh

,0
ID,
0,1350.747655
1,1406.484637
2,1122.008730
3,1142.624178
4,255.433248
...,...
3271,842.101058
3272,6303.363705
3273,2613.925000


In [ ]:
# dh = dh.reset_index()

In [ ]:
dh = dh.rename(columns={0: 'count'})
dh.to_csv("testing.csv")

In [ ]:
pd.read_csv("sample_submission.csv")

,ID,count
0,0,2176.977217
1,1,1999.479406
2,2,1984.651067
3,3,1807.153257
4,4,177.497811
...,...,...
3271,3271,1252.080186
3272,3272,3743.320334
3273,3273,3565.822523
3274,3274,1429.577997
